In [108]:
# import libraries
import langchain_google_genai
import langchain
import pinecone 

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_google_genai import ChatGoogleGenerativeAI 

In [109]:
from dotenv import load_dotenv
load_dotenv()

True

In [110]:
import os

In [111]:
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [112]:
doc = read_doc('documents/')

In [113]:
doc

[Document(metadata={'source': 'documents\\Budget_Speech.pdf', 'page': 0}, page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023'),
 Document(metadata={'source': 'documents\\Budget_Speech.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'documents\\Budget_Speech.pdf', 'page': 2}, page_content='CONTENTS \nPART-A \n Page No. \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector \n \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management \n24 \nPART B \n  \nIndirect Taxes 27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Mar

In [114]:
len(doc)

58

In [115]:
def chunk_data(docs, chunk_size=1600, chunk_overlap=50):
    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    # Split the documents
    doc_chunks = text_splitter.split_documents(docs)
    
    # Return the split documents
    return doc_chunks


In [116]:
documents = chunk_data(docs=doc)

In [117]:
documents

[Document(metadata={'source': 'documents\\Budget_Speech.pdf', 'page': 0}, page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023'),
 Document(metadata={'source': 'documents\\Budget_Speech.pdf', 'page': 2}, page_content='CONTENTS \nPART-A \n Page No. \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector \n \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management \n24 \nPART B \n  \nIndirect Taxes 27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metals  \n\uf0b7 Metals  \n\uf0

In [118]:
len(documents)

85

In [119]:
# Convert documents into a list of dictionaries with unique ID and text
documents_dict = [
    {"id": f"{d.metadata['source']}_page_{d.metadata['page']}", "text": d.page_content}
    for d in documents
]

# Now you have a list of dictionaries, where each dictionary has:
# - 'id': a unique identifier based on the document source and page number
# - 'text': the page content (text of the chunk)
print(documents_dict)


[{'id': 'documents\\Budget_Speech.pdf_page_0', 'text': 'GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023'}, {'id': 'documents\\Budget_Speech.pdf_page_2', 'text': 'CONTENTS \nPART-A \n Page No. \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector \n \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management \n24 \nPART B \n  \nIndirect Taxes 27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metals  \n\uf0b7 Metals  \n\uf0b7 Compounded Rubber  \n\uf0b7 Cigarettes  \n  \nDirect Taxes 30 

In [120]:
import os

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.environ['GEMINI_API_KEY'])
print(embeddings)

client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001B1FA873730> model='models/embedding-001' task_type=None google_api_key=SecretStr('**********') credentials=None client_options=None transport=None request_options=None


In [121]:
vector = embeddings.embed_query("hello, world!")
print(vector)
len(vector)

[0.05168594419956207, -0.030764883384108543, -0.03062233328819275, -0.02802734263241291, 0.01813093200325966, -0.0018945280462503433, 0.028477225452661514, -0.007562300190329552, 0.011064725928008556, -0.005353901535272598, 0.03884560614824295, 0.05086332932114601, -0.008304029703140259, -0.06084785610437393, -0.006586700677871704, -0.022183919325470924, 0.013262641616165638, -0.008496989496052265, -0.0006867735064588487, -0.003296870505437255, -0.0034330044873058796, 0.00962158851325512, -0.02791912667453289, -0.030381329357624054, 0.021061237901449203, 0.011688419617712498, -0.0009537383448332548, -0.07332165539264679, 0.012589256279170513, 0.05902666226029396, -0.03533710539340973, 0.017094852402806282, -0.05426131561398506, 0.006611739285290241, 0.03894328698515892, -0.05709062144160271, 0.03994591161608696, 0.009693781845271587, -0.001833937130868435, -4.905453897663392e-05, 0.024423865601420403, -0.0919656902551651, -0.04462922364473343, -0.03902028128504753, 0.04567117989063263,

768

In [122]:
pip install pinecone-client==2.2.4

Note: you may need to restart the kernel to use updated packages.


pcsk_4ZRiAj_QuzLtw4BWdiY5Uepaw3m8raUTPMbmYWPs2s84CBsnZc1EYasCSkytJP8eYMaagZ

In [123]:
pip install "pinecone[grpc]"

Note: you may need to restart the kernel to use updated packages.


In [ ]:

# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key="***********************************")


# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in documents_dict],
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings)


EmbeddingsList(
  model='multilingual-e5-large',
  data=[
    {'values': [0.0186004638671875, -0.00634002685546875, ..., -0.0465087890625, -0.0216217041015625]},
    {'values': [0.017242431640625, 0.006786346435546875, ..., -0.05487060546875, -0.0245208740234375]},
    ... (81 more embeddings) ...,
    {'values': [0.006229400634765625, -0.0311431884765625, ..., -0.007740020751953125, -0.0091400146484375]},
    {'values': [0.00012683868408203125, -0.005245208740234375, ..., -0.048309326171875, -0.04052734375]}
  ],
  usage={'total_tokens': 20709}
)


In [127]:
# Target the index where you'll store the vector embeddings
index = pc.Index("langchainvector")

# Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'
records = []
for d, e in zip(documents_dict, embeddings):
    records.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

# Upsert the records into the index
index.upsert(
    vectors=records,
    namespace="example-namespace"
)


upserted_count: 85

In [128]:

print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace': {'vector_count': 56}},
 'total_vector_count': 56}


In [129]:
# Define your query
query = "Tell me about the Infrastructure development tax"

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)


{'matches': [{'id': 'documents\\Budget_Speech.pdf_page_15',
              'metadata': {'text': '12 \n'
                                   ' \n'
                                   ' \n'
                                   ' \n'
                                   'Support to State Governments for Capital '
                                   'Investment \n'
                                   '47. I have decided to continue the 50-year '
                                   'interest free loan to state \n'
                                   'governments for one more year to spur '
                                   'investment in infrastructure and to \n'
                                   'incentivize them for complementary policy '
                                   'actions, with a significantly \n'
                                   'enhanced outlay of ` 1.3 lakh crore.   \n'
                                   'Enhancing opportunities for private '
                                   'inve